In [1]:
from fetchOddLots import fetchOddLots, genHTML
import os
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['http_proxy'] = 'http://127.0.0.1:7890'

In [2]:
df = fetchOddLots('https://www.bloomberg.com/news/articles/2023-07-10/richard-koo-on-china-s-risk-of-a-japan-style-balance-sheet-recession?srnd=oddlots','test')

https://www.bloomberg.com/news/articles/2023-07-10/richard-koo-on-china-s-risk-of-a-japan-style-balance-sheet-recession?srnd=oddlots
<a href=https://www.amazon.com/Pursued-Economy-Understanding-Overcoming-Challenging/dp/1119984270/ref=sr_1_1?crid=3QU67V3UUBT14&keywords=pursued+economy&qid=1688992673&sprefix=pursued+economy%2Caps%2C155&sr=8-1
<a href=https://www.amazon.com/Holy-Grail-Macroeconomics-Lessons-Recession/dp/0470824948/ref=sr_1_1?crid=2L0H1TNJ4BKJ0&keywords=The+Holy+Grail+of+Macroeconomics&qid=1688993172&sprefix=the+holy+grail+of+macroeconomics%2Caps%2C168&sr=8-1
<a href=https://podcasts.apple.com/us/podcast/richard-koo-explains-why-the-recovery-will-be-so-difficult/id1056200096?i=1000474211676
<a href=bbg://news/stories/RVZ7KMDWLU68


In [3]:
df

,date,title,link,content
0,2023-07-10,Richard Koo on China’s Risk of a Japan-Style B...,https://www.bloomberg.com/news/articles/2023-0...,\n <!DOCTYPE html>\n <html>\n\n <head...


In [10]:
import requests,json
from bs4 import BeautifulSoup

headers = {
    'referer': 'https://www.bloomberg.com/oddlots',
    'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1'
}
url = 'https://www.bloomberg.com/news/articles/2023-11-06/transcript-the-challenge-of-making-childcare-abundant-and-cheap'
docs = requests.get(url,headers=headers).content
doc = BeautifulSoup(docs,features='lxml')
doc1 = json.loads(doc.find('script',{'id':"__NEXT_DATA__"}).string)

In [11]:
doc1

{'props': {'pageProps': {'story': {'abstract': [],
    'archived': False,
    'adTargeting': {'adCode': 'bloomberg/markets/odd-lots/article',
     'suid': 'S3Q0C9DWX2PS01',
     'page': 'article',
     'currentResource': 'Story|S3Q0C9DWX2PS01',
     'ni': ['BUSINESS', 'EDU'],
     'tagr': [],
     'kwl': ['biz_bankofamerica',
      'biz_barclays',
      'biz_xojet',
      'biz_cartier',
      'biz_ishares',
      'biz_pimco',
      'biz_Samsung',
      'biz_wellsfargosept',
      'biz_bofasept',
      'biz_googlelist',
      'biz_googlelisttwo',
      'biz_trowe',
      'biz_atrocity',
      'biz_bp1',
      'biz_bp2',
      'biz_bp3',
      'biz_bp4',
      'biz_schwab',
      'biz_fidelity',
      'biz_ibm3',
      'biz_microsoft',
      'biz_kornferry',
      'biz_generic',
      'biz_fti',
      'biz_brandusa',
      'biz_lexus2',
      'biz_dun_bradstreet',
      'biz_United',
      'biz_BMW',
      'biz_citi1',
      'biz_mobkoimandslist',
      'biz_tdameritrade',
      'biz_boe

In [27]:




with open('html/test.html','w') as f:
    f.write('<html>'+format(doc1['props']['pageProps']['story']['body'])+'</html>')


<a href=https://www.amazon.com/Pursued-Economy-Understanding-Overcoming-Challenging/dp/1119984270/ref=sr_1_1?crid=3QU67V3UUBT14&keywords=pursued+economy&qid=1688992673&sprefix=pursued+economy%2Caps%2C155&sr=8-1
<a href=https://www.amazon.com/Holy-Grail-Macroeconomics-Lessons-Recession/dp/0470824948/ref=sr_1_1?crid=2L0H1TNJ4BKJ0&keywords=The+Holy+Grail+of+Macroeconomics&qid=1688993172&sprefix=the+holy+grail+of+macroeconomics%2Caps%2C168&sr=8-1
<a href=https://podcasts.apple.com/us/podcast/richard-koo-explains-why-the-recovery-will-be-so-difficult/id1056200096?i=1000474211676
<a href=bbg://news/stories/RVZ7KMDWLU68


'Richard Koo on China’s Risk of a Japan-Style Balance Sheet Recession'

In [21]:
formatSingle({'type': 'embed',
     'host': 'omny.fm',
     'href': 'https://omny.fm/shows/odd-lots/richard-koo-on-chinas-risk-of-japanification',
     'data': {'class': 'bbg-embed',
      'data-embed-id': 'https://omny.fm/shows/odd-lots/richard-koo-on-chinas-risk-of-japanification',
      'data-type': 'iframe',
      'href': 'https://omny.fm/shows/odd-lots/richard-koo-on-chinas-risk-of-japanification',
      'embedId': 'https://omny.fm/shows/odd-lots/richard-koo-on-chinas-risk-of-japanification',
      'type': 'iframe'},
     'iframeData': {'url': 'https://omny.fm/shows/odd-lots/richard-koo-on-chinas-risk-of-japanification',
      'rel': ['player', 'html5', 'ssl', 'playerjs', 'audio'],
      'html': '<div style="left: 0; width: 100%; height: 180px; position: relative;"><iframe src="https://omny.fm/shows/odd-lots/richard-koo-on-chinas-risk-of-japanification/embed?style=cover" style="top: 0; left: 0; width: 100%; height: 100%; position: absolute; border: 0;" allowfullscreen></iframe></div>',
      'dimension': {'width': None,
       'height': 180,
       'minWidth': None,
       'minHeight': None,
       'maxWidth': None,
       'maxHeight': None,
       'aspectRatio': None,
       '__typename': 'IframeDimension'},
      'meta': {'author': 'Odd Lots',
       'author_url': 'https://bloomberg.com/podcasts/odd_lots',
       'canonical': 'https://omny.fm/shows/odd-lots/richard-koo-on-chinas-risk-of-japanification',
       'description': "Richard Koo literally wrote the book on balance sheet recessions, or the idea that large levels of debt can weigh on future growth for years and even decades to come. Now, the Nomura Research Institute chief economist sees a similar risk emerging in China. The country has been struggling with vast levels of debt and slowing economic growth in the wake of the Covid-19 pandemic. In this episode, Koo discusses the signs he sees that a balance sheet recession is already underway as China's companies shy away from borrowing more money for future investment. He also suggests some ideas for just what China's authorities should do about it.",
       'site': 'Omny Studio',
       'title': "Richard Koo on China's Risk of 'Japanification' - Odd Lots",
       '__typename': 'IframeMeta'},
      '__typename': 'Iframe'}})

'<div style="left: 0; width: 100%; height: 180px; position: relative;"><iframe src="https://omny.fm/shows/odd-lots/richard-koo-on-chinas-risk-of-japanification/embed?style=cover" style="top: 0; left: 0; width: 100%; height: 100%; position: absolute; border: 0;" allowfullscreen></iframe></div>'